In [1]:
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np
import math

In [2]:
#Import 2018 play by play data
mylist = []
for chunk in pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv', sep=',', chunksize=20000,low_memory=False):
    mylist.append(chunk)
p18 = pd.concat(mylist, axis = 0)
del mylist

In [3]:
#Import Roster Data
rosters18 = pd.read_csv(r'rosters_2018.csv', sep = ',', low_memory = False)

In [4]:
#Count tackle assists 1-4 
tackleassists18one = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_1_player_id"] ).size()}).reset_index()
tackleassists18two = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_2_player_id"] ).size()}).reset_index()

#Merge all together into one tackle assists dataframe
tackleassists18 = pd.merge(tackleassists18one, tackleassists18two, left_on = 'assist_tackle_1_player_id', right_on = 'assist_tackle_2_player_id', how='left')

In [5]:
#Add the tackles together to get one consolidated tackle assist count for each player
tackleassists18.fillna(0)
tackleassists18['tackle_assists'] = (tackleassists18['tackles_x'] + tackleassists18['tackles_y'])
tackleassists18.drop(columns=['tackles_x', 'assist_tackle_2_player_id','tackles_y'],inplace=True)

In [6]:
#Add the number of tackle assists to the rosters18 dataframe
rosters18 = pd.merge(rosters18, tackleassists18,left_on='GSIS_ID',right_on='assist_tackle_1_player_id', how='left')
#Remove the extra player_id column
rosters18.drop(columns=['assist_tackle_1_player_id'],inplace=True)

In [7]:
#Calculate the tackle assist points for all positions as 0.5 per tackle assist
rosters18['tackle_assist_pts'] = rosters18['tackle_assists']*0.5
rosters18.head()
#Overwrite the tackle assist points for the positions that are calculated differently: CB, DE, DT
#CB tackle assist = 1 point
rosters18.loc[rosters18['Pos']=='CB',['tackle_assist_pts']] = rosters18.loc[rosters18['Pos']=='CB']['tackle_assists']
#DE tackle assist = 1 point
rosters18.loc[rosters18['Pos']=='DE',['tackle_assist_pts']] = rosters18.loc[rosters18['Pos']=='DE']['tackle_assists']
#DT tackle assist = 1.5 points
rosters18.loc[rosters18['Pos']=='DT',['tackle_assist_pts']] = rosters18.loc[rosters18['Pos']=='DT']['tackle_assists'] * 1.5

#Remove the tackle_assists series from the rosters dataframe since we only care about points
rosters18.drop(columns = ['tackle_assists'],inplace=True)
rosters18[rosters18['Pos']=='DT'].head()

Season            Player Team Pos         name     GSIS_ID  \
10    2018      Corey Peters  ARI  DT     C.Peters  00-0027686   
34    2018      Olsen Pierre  ARI  DT     O.Pierre  00-0031914   
35    2018     Pasoni Tasini  ARI  DT     P.Tasini  00-0033614   
40    2018  Robert Nkemdiche  ARI  DT  R.Nkemdiche  00-0032761   
41    2018     Rodney Gunter  ARI  DT     R.Gunter  00-0032131   

    tackle_assist_pts  
10               36.0  
34                9.0  
35                NaN  
40               16.5  
41               25.5

In [8]:
qbhit18one = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_1_player_id"] ).size()}).reset_index()
qbhit18two = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_2_player_id"] ).size()}).reset_index()

In [9]:
qbhitpts18 = pd.merge(qbhit18one, qbhit18two, left_on = 'qb_hit_1_player_id', right_on = 'qb_hit_2_player_id', how='left')
qbhitpts18.fillna(0, inplace=True)

In [10]:
qbhitpts18['qb_hit_pts'] = qbhitpts18['qb_hit_x'] + qbhitpts18['qb_hit_y']

In [11]:
rosters18 = pd.merge(rosters18, qbhitpts18[['qb_hit_pts','qb_hit_1_player_id']],left_on='GSIS_ID',right_on='qb_hit_1_player_id', how='left')

In [12]:
rosters18.drop(['qb_hit_1_player_id'], axis=1,inplace=True)

In [13]:
#Count tackle assists 1-4 
solotackle18one = pd.DataFrame({'solo_tackles' : p18.groupby( [ "solo_tackle_1_player_id"] ).size()}).reset_index()
solotackle18two = pd.DataFrame({'solo_tackles' : p18.groupby( [ "solo_tackle_2_player_id"] ).size()}).reset_index()

#Merge all together into one tackle assists dataframe
solotackle18 = pd.merge(solotackle18one, solotackle18two, left_on = 'solo_tackle_1_player_id', right_on = 'solo_tackle_2_player_id', how='left')
solotackle18.fillna(0, inplace=True)

In [14]:
#Add the tackles together to get one consolidated tackle assist count for each player
solotackle18.fillna(0, inplace=True)
solotackle18['solo_tackles'] = (solotackle18['solo_tackles_x'] + solotackle18['solo_tackles_y'])
solotackle18.drop(columns=['solo_tackles_x', 'solo_tackle_2_player_id','solo_tackles_y'],inplace=True)

In [15]:
#Add the number of tackle assists to the rosters18 dataframe
rosters18 = pd.merge(rosters18, solotackle18,left_on='GSIS_ID',right_on='solo_tackle_1_player_id', how='left')
#Remove the extra player_id column
rosters18.drop(columns=['solo_tackle_1_player_id'],inplace=True)

In [16]:
#Calculate the tackle points for all positions as 1 per tackle
rosters18['tackle_pts'] = rosters18['solo_tackles']
rosters18.head()
#Overwrite the tackle assist points for the positions that are calculated differently: CB, DE, DT
#CB tackle assist = 1 point
rosters18.loc[rosters18['Pos']=='CB',['tackle_pts']] = rosters18.loc[rosters18['Pos']=='CB']['solo_tackles']
#DE tackle assist = 2 points
rosters18.loc[rosters18['Pos']=='DE',['tackle_pts']] = rosters18.loc[rosters18['Pos']=='DE']['solo_tackles'] * 2
#DT tackle assist = 2.5 points
rosters18.loc[rosters18['Pos']=='DT',['tackle_pts']] = rosters18.loc[rosters18['Pos']=='DT']['solo_tackles'] * 2.5

#Remove the tackle_assists series from the rosters dataframe since we only care about points
rosters18.drop(columns = ['solo_tackles'],inplace=True)
rosters18[rosters18['Pos']=='DT'].head()

Season            Player Team Pos         name     GSIS_ID  \
10    2018      Corey Peters  ARI  DT     C.Peters  00-0027686   
34    2018      Olsen Pierre  ARI  DT     O.Pierre  00-0031914   
35    2018     Pasoni Tasini  ARI  DT     P.Tasini  00-0033614   
40    2018  Robert Nkemdiche  ARI  DT  R.Nkemdiche  00-0032761   
41    2018     Rodney Gunter  ARI  DT     R.Gunter  00-0032131   

    tackle_assist_pts  qb_hit_pts  tackle_pts  
10               36.0         4.0        67.5  
34                9.0         1.0        15.0  
35                NaN         NaN         NaN  
40               16.5         7.0        52.5  
41               25.5         9.0        67.5

In [17]:
fumble18 = pd.DataFrame({'fumbles' : p18.groupby( [ "fumbled_1_player_id"] ).size()}).reset_index()
fumble18.fillna(0, inplace=True)

In [18]:
fumble18['fumble_pts'] = fumble18['fumbles'] * -5

In [19]:
rosters18 = pd.merge(rosters18, fumble18[['fumble_pts','fumbled_1_player_id']],left_on='GSIS_ID',right_on='fumbled_1_player_id', how='left')

In [20]:
rosters18.drop(['fumbled_1_player_id'], axis=1,inplace=True)

In [21]:
recoverydf = p18[["fumble_recovery_1_player_id", "fumble_recovery_1_yards"]]
recoverydf.groupby(["fumble_recovery_1_player_id"]).sum()

fumble_recovery_1_yards
fumble_recovery_1_player_id                         
00-0020531                                       0.0
00-0021140                                       0.0
00-0022161                                      43.0
00-0022787                                       0.0
00-0022803                                       0.0
00-0022924                                       0.0
00-0022942                                       0.0
00-0023259                                       0.0
00-0023436                                       0.0
00-0023448                                       0.0
00-0023459                                       0.0
00-0023645                                       0.0
00-0023682                                       0.0
00-0024030                                       0.0
00-0024061                                       0.0
00-0024348                                       0.0
00-0025418                                       0.0
00-0025446                                       0.0
00-0025473                                       0.0
00-0025828                                       0.0
00-0025900                                       0.0
00-0026143                                       0.0
00-0026158                                       0.0
00-0026160                                      47.0
00-0026176                                       0.0
00-0026190                                       0.0
00-0026327                                       0.0
00-0026498                                       0.0
00-0026544                                       0.0
00-0026839                                       0.0
...                                              ...
00-0034751                                       0.0
00-0034756                                       0.0
00-0034763                                       0.0
00-0034766                                       0.0
00-0034767                                       0.0
00-0034774                                       0.0
00-0034777                                       7.0
00-0034778                                       0.0
00-0034781                                       0.0
00-0034782                                       0.0
00-0034785                                       0.0
00-0034796                                       0.0
00-0034798                                       0.0
00-0034805                                       0.0
00-0034811                                       0.0
00-0034815                                       0.0
00-0034827                                       0.0
00-0034832                                       4.0
00-0034834                                      19.0
00-0034837                                       0.0
00-0034843                                       0.0
00-0034846                                       0.0
00-0034847                                       0.0
00-0034855                                       0.0
00-0034856                                      29.0
00-0034857                                       0.0
00-0034858                                       0.0
00-0034860                                       0.0
00-0034869                                       0.0
00-0034881                                       0.0

[434 rows x 1 columns]

In [22]:
rosters18 = pd.merge(rosters18, recoverydf[["fumble_recovery_1_player_id","fumble_recovery_1_yards"]],left_on='GSIS_ID',right_on="fumble_recovery_1_player_id", how='left')

In [23]:
rosters18['recoveryyard_pts'] = rosters18["fumble_recovery_1_yards"] * 0.15

In [24]:
rosters18.drop(["fumble_recovery_1_player_id","fumble_recovery_1_yards"], axis=1,inplace=True)

In [25]:
recovery18 = pd.DataFrame({'recovery' : p18.groupby( [ "fumble_recovery_1_player_id"] ).size()}).reset_index()
recovery18.fillna(0, inplace=True)

In [26]:
recovery18['recovery_pts'] = recovery18['recovery'] * 5

In [27]:
rosters18 = pd.merge(rosters18, recovery18[['recovery_pts','fumble_recovery_1_player_id']],left_on='GSIS_ID',right_on='fumble_recovery_1_player_id', how='left')

In [28]:
rosters18.drop(['fumble_recovery_1_player_id'], axis=1,inplace=True)

In [29]:
intercept18 = pd.DataFrame({'interceptions' : p18.groupby( [ "interception_player_id"] ).size()}).reset_index()
intercept18.fillna(0, inplace=True)

In [30]:
intercept18['interceptionscaught_pts'] = intercept18['interceptions'] * 6

In [31]:
intercept18 = pd.merge(rosters18, intercept18[['interceptionscaught_pts',"interception_player_id"]],left_on='GSIS_ID',right_on="interception_player_id", how='left')

In [32]:
intercept18.drop(["interception_player_id"], axis=1,inplace=True)

In [33]:
passdefend18one = pd.DataFrame({'pass_defend' : p18.groupby( [ "pass_defense_1_player_id"] ).size()}).reset_index()
passdefend18two = pd.DataFrame({'pass_defend' : p18.groupby( [ "pass_defense_2_player_id"] ).size()}).reset_index()

In [34]:
passdefences18 = pd.merge(passdefend18one, passdefend18two, left_on = 'pass_defense_1_player_id', right_on = 'pass_defense_2_player_id', how='left')

In [35]:
passdefences18.fillna(0)
passdefences18['pass_defences'] = (passdefences18['pass_defend_x'] + passdefences18['pass_defend_y']) * 3
passdefences18.drop(columns=['pass_defend_x', 'pass_defense_2_player_id','pass_defend_y'],inplace=True)
passdefences18.head()

pass_defense_1_player_id  pass_defences
0               00-0021140            NaN
1               00-0022161            NaN
2               00-0022247            NaN
3               00-0023259            NaN
4               00-0023368            NaN

In [36]:
rosters18 = pd.merge(rosters18, passdefences18,left_on='GSIS_ID',right_on='pass_defense_1_player_id', how='left')
rosters18.drop(columns=['pass_defense_1_player_id'],inplace=True)
rosters18.head()

Season            Player Team Pos        name     GSIS_ID  \
0    2018    Antoine Bethea  ARI  FS    A.Bethea  00-0024421   
1    2018     Benson Mayowa  ARI  DE    B.Mayowa  00-0030380   
2    2018  Brandon Williams  ARI  CB  B.Williams  00-0032769   
3    2018       Budda Baker  ARI  SS     B.Baker  00-0033890   
4    2018       Budda Baker  ARI  SS     B.Baker  00-0033890   

   tackle_assist_pts  qb_hit_pts  tackle_pts  fumble_pts  recoveryyard_pts  \
0               13.5         6.0        93.0         NaN               NaN   
1                9.0        11.0        58.0         NaN               0.0   
2                NaN         NaN        10.0         NaN               NaN   
3               15.5         3.0        71.0         NaN               5.4   
4               15.5         3.0        71.0         NaN              -0.3   

   recovery_pts  pass_defences  
0           NaN            NaN  
1           5.0            NaN  
2           NaN            NaN  
3          10.0            NaN  
4          10.0            NaN

In [37]:
rosters18.fillna(0, inplace=True)
rosters18.to_excel('output.xlsx', engine='xlsxwriter')